In [ ]:
# default_exp core

# Core

> Basic functionality for unpickling objects

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
import shutil

In [ ]:
#export
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.basics import setify
from fastcore.meta import delegates

from importlib import import_module

import os
import pickle

In [ ]:
#export
def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=True, folders=None, followlinks=True):
    "Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified."
    path = Path(path)
    folders=L(folders)
    extensions = setify(extensions)
    extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path, followlinks=followlinks)): # returns (dirpath, dirnames, filenames)
            if len(folders) !=0 and i==0: d[:] = [o for o in d if o in folders]
            else:                         d[:] = [o for o in d if not o.startswith('.')]
            if len(folders) !=0 and i==0 and '.' not in folders: continue
            res += _get_files(p, f, extensions)
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        res = _get_files(path, f, extensions)
    return L(res)

In [ ]:
#export
def _try_import(path, name):
    for file in get_files(path, extensions='.py'):
        try:
            lib_name = str(file).split(path.name)[1]
            import_statement = f'{path.name}'
            for item in lib_name.split('/'):
                if len(item.split('.')) > 1:
                    item = item.split('.')[0]
                import_statement += f'{item}.'
            mod = import_module(import_statement[:-1], name)
            g = globals()
            g[name] = getattr(mod, name)
            return
        except:
            raise ModuleNotFoundError(f"Could not find {name} in any modules from {path.name}")
    raise ModuleNotFoundError(f"Could not find {name} in any modules from {path.name}")

In [ ]:
#hide
os.mkdir('lib')
tmp = open('lib/tmp.py', 'w')
tmp.write('def foo(a, b): return a, b')
tmp.close()
_try_import(Path('lib'), 'foo')
test_eq(foo('x', 'y'), ('x', 'y'))
shutil.rmtree('lib')
g = globals()
g.pop('foo');
del g

In [ ]:
#hide
with ExceptionExpected(ModuleNotFoundError):
    _try_import(Path('lib'), 'foo')

In [ ]:
#export
class UnpicklerModule(pickle.Unpickler):
    "Custom unpickler class for getting classes from a custom library"
    @delegates(pickle.Unpickler)
    def __init__(self, lib_path:Path=Path('.'), **kwargs):
        store_attr()
        super().__init__(**kwargs)
    
    def find_class(self, module, name):
        try:
            return super().find_class(module, name)
        except:
            _try_import(self.lib_path, module, name)
            return super().find_class(module, name)